In [ ]:
import numpy as np
import pandas as pd
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.linear_model import LinearRegression
import pickle

# Определяем кастомный трансформер
class MyTransformer(TransformerMixin, BaseEstimator):
    '''Шаблон кастомного трансформера'''

    def __init__(self):
        '''Инициализация параметров, не зависящих от данных.'''
        pass

    def fit(self, X, y=None):
        '''Обучение трансформера.'''
        return self

    def transform(self, X):
        '''Действия с данными.'''
        # Преобразуем X в numpy.ndarray, если это pandas.DataFrame
        if isinstance(X, pd.DataFrame):
            X = X.to_numpy()

        # Создаём новый столбец как произведение первых трёх столбцов
        new_column = X[:, 0] * X[:, 1] * X[:, 2]
        # Изменяем размер на (n_rows, 1)
        new_column = new_column.reshape(X.shape[0], 1)
        # Добавляем столбец в матрицу измерений
        X = np.append(X, new_column, axis=1)
        return X

# Пример данных
X = pd.DataFrame(np.random.rand(442, 10))  # Создаем DataFrame для примера
y = np.random.rand(442)

# Инициализируем объект класса MyTransformer
custom_transformer = MyTransformer()
# Вызываем метод fit (формально)
custom_transformer.fit(X)
# Трансформируем исходные данные
X_transformed = custom_transformer.transform(X)
print('Shape before transform: {}'.format(X.shape))
print('Shape after transform: {}'.format(X_transformed.shape))

# Создаём пайплайн
pipe = Pipeline([
    ('FeatureEngineering', MyTransformer()),
    ('Scaling', MinMaxScaler()),
    ('FeatureSelection', SelectKBest(f_regression, k=5)),
    ('Linear', LinearRegression())
])

# Обучаем пайплайн
pipe.fit(X, y)

# Сериализуем pipeline и записываем результат в файл
with open('my_new_pipeline.pkl', 'wb') as output:
    pickle.dump(pipe, output)

print("Pipeline успешно создан и сохранен.")

In [ ]:
import numpy as np
import pickle

# Загрузка pipeline из файла
with open('my_new_pipeline.pkl', 'rb') as input_file:
    loaded_pipe = pickle.load(input_file)

# Вектор для предсказания
features = np.array([[0.00538306, -0.04464164, 0.05954058, -0.05616605, 0.02457414,
                      0.05286081, -0.04340085, 0.05091436, -0.00421986, -0.03007245]])

# Предсказание
prediction = loaded_pipe.predict(features)

# Округление результата
rounded_prediction = int(round(prediction[0]))

print("Предсказанное значение целевой переменной:", rounded_prediction)